<a href="https://colab.research.google.com/github/adeledouin/Collab/blob/main/test_code_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collab config TPU

In [1]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl


# Import Projet

In [2]:
import sys
import os
import torch 

In [3]:
py_file_location = "drive/MyDrive/knit_quakes_forecast/knit_005/"
sys.path.append(os.path.abspath(py_file_location))


from classConfig import ConfigData, ConfigPred
import Config_data
import Config_pred
from classPlot import ClassPlot
from Datas.classStat import Histo, Stat
from Datas.classSignal import SignalForce, VariationsScalar
from Datas.classEvent import ForceEvent
# from sub_train_val_test import train_model, test_model
from Datas.classData import find_nbseq, CreateDataScalar
from Datas.classLoader import DataSetTIMESCALAR, DataSetFalseData
from Datas.sub_createdata import create_sequences_field, \
    create_generator_field, create_generator_false_data
from Datas.classDataParallel import DataParallel
from sub_train_val_test import train_model, test_model
from Models.classLoss import MultiGPULossCompute, SimpleLossCompute
from Models.classModel import def_nn_models

# Run

In [4]:
import numpy as np
import torch
import warnings
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
from os import listdir

# # PyTorch/XLA GPU Setup (only if GPU runtime)
# if os.environ.get('COLAB_GPU', '0') == '1':
#   os.environ['GPU_NUM_DEVICES'] = '1'
#   os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda/'

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu

warnings.filterwarnings("ignore")


In [5]:

# %% ################### Args ##################################
remote = False

date = '210617'
ref_tricot = 'knit005_'
n_exp = 'mix_'
version_data = 'v2'
version_pred = 'v3'
sub_version = ''
model = 19

train = True
checkpoint = ''

transfert = False
transfert_date = '210616'
transfert_pred = ''
transfert_sub_version = ''
transfert_trainsize = 100000

epochs = 5
trainsize = 1000000
cuda_device = ''
num_worker = 4
num_cores = 8 if os.environ.get('TPU_NAME', None) else 1
log_step = 20
metrics_debug = False


class Args:
    def __init__(self):
        self.remote = remote
        self.cuda = False
        self.vpred = version_pred
        self.subv = sub_version
        self.pred = 'scalar'
        self.train = train
        self.verif_stats_output = False
        self.checkpoint = checkpoint
        self.epoch = epochs
        self.trainsize = trainsize
        self.cuda_device = cuda_device
        self.transfert = transfert
        self.transfert_date = transfert_date
        self.transfert_pred = transfert_pred
        self.transfert_sub_version = transfert_sub_version
        self.transfert_trainsize = transfert_trainsize
        self.num_workers = num_worker
        self.num_cores = num_cores
        self.log_step = log_step
        self.metric_debug = metrics_debug

args = Args()

if args.cuda:
    if type(args.cuda_device).__name__ == 'str':
        device = torch.device(args.cuda_device)
    else:
        device = args.cuda_device
else:
    device = "cpu"

NAME_EXP = ref_tricot + n_exp + version_data
config_data = ConfigData(Config_data.exp[NAME_EXP])

NAME_EXP = ref_tricot + version_pred
if args.pred == 'scalar':
    version_exp = Config_pred.exp_scalar[NAME_EXP]
    config_pred = ConfigPred(Config_pred.exp_scalar[NAME_EXP], config_data)
else:
    version_exp = Config_pred.exp_img[NAME_EXP]
    config_pred = ConfigPred(Config_pred.exp_img[NAME_EXP], config_data)

config_pred.set_model_attribute(model)
if args.pred == 'scalar':
    model_spec = config_pred.spec_model_scalar()
else:
    model_spec = config_pred.spec_model_img()

histo = Histo(config_data)
plot = ClassPlot(args.remote, histo)



In [6]:
print(config_pred.global_path_load)

print(config_pred.global_path_save)

# os.chdir("/content/drive/MyDrive/knit_quakes_forecast/knit_005/input/version2/flu_rsc_NN")
# !ls

a = np.load('/content/drive/MyDrive/knit_quakes_forecast/knit_005/input/version2/' + 'flu_rsc_NN/flu_rsc_train.npy')

/content/drive/MyDrive/knit_quakes_forecast/knit_005/input/version2/
/content/drive/MyDrive/knit_quakes_forecast/knit_005/output/version2/


In [7]:

# %% ################### Create sequences ##################################

print('------ create data sequences ------')
signal_flu = [SignalForce(config_data, 'flu_rsc', NN) for NN in config_pred.NN_data]
signalevent = [ForceEvent(config_data, signal_flu[i].f, signal_flu[i].ext, signal_flu[i].t,
                          'flu_rsc', config_pred.NN_data[i], Sm=False) for i in
               range(np.size(config_pred.NN_data))]

nb_seq = [
    find_nbseq(config_pred, signal_flu[0], config_pred.seq_size, config_pred.futur, config_pred.overlap_step,
               trainsize) if trainsize is None else trainsize,
    find_nbseq(config_pred, signal_flu[1], config_pred.seq_size, config_pred.futur, config_pred.overlap_step,
               trainsize),
    find_nbseq(config_pred, signal_flu[2], config_pred.seq_size, config_pred.futur, config_pred.overlap_step,
               trainsize)]

------ create data sequences ------
/content/drive/MyDrive/knit_quakes_forecast/knit_005/input/version2/flu_rsc_NN/
/content/drive/MyDrive/knit_quakes_forecast/knit_005/input/version2/flu_rsc_NN/
/content/drive/MyDrive/knit_quakes_forecast/knit_005/input/version2/flu_rsc_NN/


In [8]:
def train_model(args, device, nb_seq):
    torch.manual_seed(1)

    def get_dataset():
        # print('avant train nb seq = {}'.format(nb_seq))
        train_dataset = DataSetTIMESCALAR(config_data, config_pred, remote, plot, histo, nb_seq.copy(), signal_flu,
                                          signalevent, 0)
        # print('avant val nb seq = {}'.format(nb_seq))
        val_dataset = DataSetTIMESCALAR(config_data, config_pred, remote, plot, histo, nb_seq.copy(), signal_flu,
                                        signalevent,
                                        1)
        # print('avant test nb seq = {}'.format(nb_seq))
        test_dataset = DataSetTIMESCALAR(config_data, config_pred, remote, plot, histo, nb_seq.copy(), signal_flu,
                                         signalevent,
                                         2)

        return train_dataset, val_dataset, test_dataset

    SERIAL_EXEC = xmp.MpSerialExecutor()
    # Using the serial executor avoids multiple processes
    # to download the same data.
    train_dataset, val_dataset, test_dataset = SERIAL_EXEC.run(get_dataset)


    if config_pred.output_type == 'class':
        if not config_pred.equipro:
            config_pred.set_weight_classes(1 / train_dataset.prop)
            print(train_dataset.prop, 1 / train_dataset.prop)
    nb_seq = train_dataset.nb_seq.copy()

    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config_pred.batch_size,
        sampler=train_sampler,
        num_workers=args.num_workers,
        drop_last=True)
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=config_pred.batch_size,
        shuffle=False,
        num_workers=args.num_workers,
        drop_last=True)
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=config_pred.batch_size,
        shuffle=False,
        num_workers=args.num_workers,
        drop_last=True)

    # %% ################### Informations ##################################

    if args.pred == 'scalar':
        num_test = ('{}_{}_m{}_{}seq{}').format(date, version_pred, model_spec['magic_number'], nb_seq[0], sub_version)
    else:
        num_test = ('{}_{}_m{}_{}seq{}').format(date, version_pred, model_spec['magic_number'], nb_seq[0], sub_version)

    save_best_acc = config_pred.global_path_save + 'net/{}/best_acc_{}.tar'.format(args.pred, num_test)
    save_best_loss = config_pred.global_path_save + 'net/{}/best_loss_{}.tar'.format(args.pred, num_test)
    save_dict_version = config_pred.global_path_save + 'net/{}/config_pred_{}'.format(args.pred, num_test)
    save_dict_model = config_pred.global_path_save + 'net/{}/model_{}'.format(args.pred, num_test)

    print('------ {} on model {} ------'.format(NAME_EXP, model_spec['magic_number']))
    print('run on cuda : {} ; cuda device : {}'.format(args.cuda, device))
    if args.train:
        print('on va train {} on {} epochs with training size {}'.format(model_spec['model_name'], args.epoch, nb_seq[0]))
    else:
        print('on va test {}'.format(model_spec['model_name']))
    print('pred with {} past on futur window of {}'.format(config_pred.seq_size, config_pred.futur))
    print('avec {} seqs dans val, {} seqs dans test'.format(nb_seq[1], nb_seq[2]))


    # %% ################### NN model ##################################

    print('------ create model ------')
    np.save(save_dict_version, version_exp)
    np.save(save_dict_model, model_spec)


    if args.pred == 'scalar':
        def_nn = def_nn_models(args, device, model_spec['model_name'], config_pred.output_type,
                               model_spec['layers_param'], model_spec['opti_param'], model_spec['criterion_param'],
                               config_pred.batch_size, config_pred.seq_size, config_pred.channel, None, None,
                               config_pred.output_shape)
    else:
        def_nn = def_nn_models(args, device, model_spec['model_name'], config_pred.output_type,
                               model_spec['layers_param'], model_spec['opti_param'], model_spec['criterion_param'],
                               config_pred.batch_size, config_pred.seq_size, config_pred.channel, sw, sc,
                               config_pred.output_shape)

    model, optimizer, scheduler, loss_criterion = def_nn.NN_model()

    model, optimizer = def_nn.recup_from_checkpoint(config_pred, args, model_spec, model, optimizer, num_test, save_best_acc, save_best_loss)

    if type(args.cuda_device).__name__ == 'list':
        model = DataParallel(model, device_ids=device)

    # Only instantiate model weights once in memory.
    WRAPPED_MODEL = xmp.MpModelWrapper(model)

    # Scale learning rate to num cores
    learning_rate = model_spec['opti_param']['lr'] * xm.xrt_world_size()

    # Get loss function, optimizer, and model
    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)


    # %% ################### Training ##################################
    # Train and eval loops
    from Train_loop_collab import train_loop_fn, test_loop_fn

    accuracy = 0.0
    data, pred, target = None, None, None
    for epoch in range(1, args.epoch + 1):
        para_loader = pl.ParallelLoader(train_loader, [device])
        train_loop_fn(args, config_pred, para_loader.per_device_loader(device), model, optimizer, loss_criterion)
        xm.master_print("Finished training epoch {}".format(epoch))

        para_loader = pl.ParallelLoader(test_loader, [device])
        accuracy, data, pred, target = test_loop_fn(para_loader.per_device_loader(device), model)
        if args.metrics_debug:
            xm.master_print(met.metrics_report(), flush=True)

    return accuracy, data, pred, target

# Start training processes
def _mp_fn(rank, args, device, nb_seq):
  torch.set_default_tensor_type('torch.FloatTensor')
  accuracy, data, pred, target = train_model(args, device, nb_seq)
  if rank == 0:
    # Retrieve tensors that are on TPU core 0 and plot.

    # %% ################### plot callback ##################################

    save_metric = config_pred.global_path_save + 'callback/{}/metric'.format(args.pred)

    y_target = target.cpu()

    y_pred = pred.cpu()


    def classes_scores(config_pred, y_target, repport=False):
        target = np.zeros((np.size(y_target), config_pred.output_shape))
        pred = np.zeros((np.size(y_target), config_pred.output_shape))

        for i in range(np.size(y_target)):
            target[i, int(y_target[i])] = 1
            pred[i, int(y_pred[i])] = 1

        print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_target, y_pred)))

        print('Micro Precision: {:.2f}'.format(precision_score(y_target, y_pred, average='micro')))
        print('Micro Recall: {:.2f}'.format(recall_score(y_target, y_pred, average='micro')))
        print('Micro F1-score: {:.2f}\n'.format(f1_score(y_target, y_pred, average='micro')))

        print('Macro Precision: {:.2f}'.format(precision_score(y_target, y_pred, average='macro')))
        print('Macro Recall: {:.2f}'.format(recall_score(y_target, y_pred, average='macro')))
        print('Macro F1-score: {:.2f}\n'.format(f1_score(y_target, y_pred, average='macro')))

        print('Weighted Precision: {:.2f}'.format(precision_score(y_target, y_pred, average='weighted')))
        print('Weighted Recall: {:.2f}'.format(recall_score(y_target, y_pred, average='weighted')))
        print('Weighted F1-score: {:.2f}'.format(f1_score(y_target, y_pred, average='weighted')))

        if repport:
            print('\nClassification Report\n')
            print(classification_report(y_target, y_pred,
                                        target_names=['Class {}'.format(i) for i in range(config_pred.output_shape)]))


    def make_grid_classes(classes_edges):
        grid = np.zeros(np.size(classes_edges))
        for i in range(np.size(classes_edges)):
            grid[i] = classes_edges[i]
        return grid


    def accuracy_value_dispach(y_value, y_target, y_pred):
        g_value = []
        w_value = []

        for i in range(np.size(y_value)):
            if y_pred[i] == y_target[i]:
                g_value.append(y_value[i])
            else:
                w_value.append(y_value[i])

        return np.asarray(g_value), np.asarray(w_value)


    if config_pred.output_type == 'class':
        from Datas.classStat import Stat

        nbclasses = config_pred.output_shape
        print('#-------------classes--------------#')

        classes_scores(config_pred, y_target, y_pred, repport=True)

        conf = np.asarray(confusion_matrix(y_target, y_pred))

        X_conf = np.arange(nbclasses)
        yname = 'quantité pred'
        ysave = 'qt_pred'
        fig, ax = plot.belleFigure('${}$'.format('classe'), '${}$'.format(yname), nfigure=None)
        norm = np.asarray([np.sum(conf[:, i]) for i in range(nbclasses)])
        for i in range(nbclasses):
            ax.plot(X_conf, (conf[i, :] / norm) * 100, '.', label='classe {}'.format(i))
        # plot.plt.xscale('log')
        # plot.plt.yscale('log')
        save = save_metric + '_qt_pred'
        plot.fioritures(ax, fig, title='{}'.format(ysave), label=True, grid=None, save=save)

        # print('\nClassification Report\n')
        # print(classification_report(y_target, y_pred,
        #                             target_names=['Class {}'.format(i) for i in range(config_pred.output_shape)]))


        conf_norm = np.zeros_like(conf, dtype=float)
        for i in range(nbclasses):
            conf_norm[i, :] = (conf[i, :] / norm) * 100



    else:
        MAE = np.mean(np.abs(y_target - y_pred))
        MSE = np.mean((y_target - y_pred) ** 2)
        M4E = np.mean((y_target - y_pred) ** 4)
        M6E = np.mean((y_target - y_pred) ** 6)

        print(
            '\tMAE: {:.6f} |\tMSE: {:.6f} |\tM4E: {:.6f} |\tM6E: {:.6f}'.format(
                MAE, MSE, M4E, M6E))

        fig, ax = plot.belleFigure('${}$'.format('pts'), '${}$'.format('y'), nfigure=None)
        ax.plot(np.arange(y_target.size), y_target, 'b.', label='target')
        ax.plot(np.arange(y_pred.size), y_pred, 'r.', label='pred')
        save = save_metric + '_y_time_plot'
        plot.fioritures(ax, fig, title=None, label=True, grid=None, save=save)

        fig, ax = plot.belleFigure('${}$'.format('y_{target}'), '${}$'.format('y_{pred}'), nfigure=None)
        ax.plot(y_target, y_pred, '.', label='')
        save = save_metric + '_y_target_vs_pred'
        plot.fioritures(ax, fig, title=None, label=None, grid=None, save=save)


In [9]:
xmp.spawn(_mp_fn, args=(args, device,nb_seq, ), nprocs=args.num_cores,
          start_method='fork')

[0.348 0.326 0.326] [2.87356322 3.06748466 3.06748466]
------ knit005_v3 on model 5019 ------
run on cuda : False ; cuda device : cpu
on va train ResNet on 5 epochs with training size 1000000
pred with 256 past on futur window of 20
avec 200000 seqs dans val, 200000 seqs dans test
------ create model ------
ResNet


Exception in device=TPU:0: 'module' object is not callable
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 329, in _mp_start_fn
    _start_fn(index, pf_cfg, fn, args)
  File "/usr/local/lib/python3.7/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 323, in _start_fn
    fn(gindex, *args)
  File "<ipython-input-8-e973019b9768>", line 134, in _mp_fn
    accuracy, data, pred, target = train_model(args, device, nb_seq)
  File "<ipython-input-8-e973019b9768>", line 95, in train_model
    model, optimizer, scheduler, loss_criterion = def_nn.NN_model()
  File "/content/drive/MyDrive/knit_quakes_forecast/knit_005/Models/classModel.py", line 216, in NN_model
    criterion = criterion_cross_entropy_loss(torch(self.criterion_param['weight']))
TypeError: 'module' object is not callable


ProcessExitedException: ignored